## MNIST Handwritten Numbers detection via CNN using Keras,TensorFlow 2.0

In [ ]:
#Import Modules
import tensorflow as tf
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import copy

#Dataset.
from tensorflow.keras.datasets import mnist
#One-hot encoding.
from tensorflow.keras.utils import to_categorical
#Feedforward sequential network with no feedback loop.
from tensorflow.keras.models import Sequential
#Layers.
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
#Optimizer.
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
#Model loader.
from tensorflow.keras.models import load_model



In [ ]:
#load the MNIST handwritten number dataset.
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print(x_train.shape)
print(x_test.shape)
print(x_train[0].shape)

In [ ]:
#Display random samples from training set.

#pyplot subplot nrows,ncols,index->3,3,1
subplot_num=331
for index in range(0,5):
    random_num=np.random.randint(0,len(x_train))
    image=x_train[random_num]
    plt.subplot(subplot_num)
    plt.imshow(image,cmap='gray')
    subplot_num=subplot_num+1

In [ ]:
#Preprocess the data.
#Convert the image shape for Keras.
#Convert labels to one-hot encoded representation.
#Normalize the data.

#Convert for Keras.
img_rows=x_train[0].shape[0]
img_cols=x_train[0].shape[1]
input_img_shape=(img_rows,img_cols,1)

#Reshape.
x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)

#One Hot encoding.
print('Before one-hot encoding',y_train.shape[0])

y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

print("After one-hot encoding",y_train.shape[0],y_train.shape[1])
num_classes=y_train.shape[1]
num_pixels=x_train.shape[0]*x_train.shape[1]

#Normalize the data.
print(x_test.dtype)
print(x_train.dtype)
x_train=x_train.astype('float32')
x_test=x_test.astype('float32')

x_train=x_train/255.0
x_test=x_test/255.0



In [ ]:
#Build the model.

model=Sequential()

#First layer.
#padding=0, stride=1
#(nxn)*(f*f) =[((n+2p-f)/s) +1, ((n+2p-f)/s) +1]
# (28x28x1)*(3x3x32)= (26x26x32)
model.add(Conv2D(32, kernel_size=(3,3), activation='relu',input_shape=input_img_shape))

#Second layer.(24x24x64)
model.add(Conv2D(64,(3,3),activation='relu'))

#Max-Pooling.(12x12x64)
model.add(MaxPooling2D(pool_size=(2,2)))

#Dropout
model.add(Dropout(0.25))

#Flatten 9216x1
model.add(Flatten())

#Dense Layer (1x128)
model.add(Dense(128,activation='relu'))

#Dropout.
model.add(Dropout(0.5))

#Final layer 
model.add(Dense(num_classes,activation='softmax'))

#Compile the model.(SGD, LR=0.01, loss=CCE)
model.compile(loss='categorical_crossentropy', optimizer=SGD(0.01), metrics=['accuracy'])

print(model.summary())

In [ ]:
#Training the model
batch=5
epochs=10

history=model.fit(x_train,y_train,batch_size=batch,epochs=epochs,verbose=1,validation_data=(x_test,y_test))


In [ ]:
#Evaluate the model
score=model.evaluate(x_test,y_test,verbose=1)

print('Test accuracy',score[1])
print('Test loss',score[0])

In [ ]:
#Plot loss.

history_dict=history.history

loss_values=history_dict['loss']
#validation loss
val_loss=history_dict['val_loss']

epochs=range(1,len(loss_values)+1)

line1=plt.plot(epochs,val_loss,label='Validation loss')
line2=plt.plot(epochs,loss_values,label='Training loss')
plt.setp(line1,linewidth=2.0,marker='+', markersize=10.0)
plt.setp(line2,linewidth=2.0,marker='3',markersize=10.0)
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
#Plot accuracy
history_dict = history.history

acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(loss_values) + 1)

line1 = plt.plot(epochs, val_acc_values, label='Validation/Test Accuracy')
line2 = plt.plot(epochs, acc_values, label='Training Accuracy')
plt.setp(line1, linewidth=2.0, marker = '+', markersize=10.0)
plt.setp(line2, linewidth=2.0, marker = '4', markersize=10.0)
plt.xlabel('Epochs') 
plt.ylabel('Accuracy')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
#Save the model.
model.save('mnist_number_trained_model.h5')



In [ ]:
#load the saved model
classifier=load_model('mnist_number_trained_model.h5')

In [ ]:
#Test on test data.
test_image=x_test[3]
print(test_image.shape)
#Reshape for Keras.
test_image=test_image.reshape(1,28,28,1)

#res=classifier.predict_classes(test_image,1,verbose=0)
#res=str(res[0])
#print(res)

res=classifier.predict(test_image)
print(res)
res=np.argmax(res)
print(res)

test_image=test_image.reshape(28,28)
plt.imshow(test_image,cmap='gray')
print('Prediction',res)

In [ ]:
cap = cv.VideoCapture(0)

while True:

    ret, frame = cap.read()
    
    #define region of interest
    roi = frame[100:400, 320:620]
    cv.imshow('roi', roi)
    roi = cv.cvtColor(roi, cv.COLOR_BGR2GRAY)
    roi = cv.resize(roi, (28, 28), interpolation = cv.INTER_AREA)
    
    cv.imshow('roi sacled and gray', roi)
    copy = frame.copy()
    cv.rectangle(copy, (320, 100), (620, 400), (255,0,0), 5)
    
    roi = roi.reshape(1,28,28,1) 

    result = classifier.predict(roi)
    result =np.argmax(result)
    #print(result)
    result=str(result)[0]
    cv.putText(copy, result, (300 , 100), cv.FONT_HERSHEY_COMPLEX, 2, (0, 255, 0), 2)
    cv.imshow('frame', copy)    
    
    if cv.waitKey(1) == 13: #13 is the Enter Key
        break
        
cap.release()
cv.destroyAllWindows() 